In [0]:
%sql
use hive_metastore.default

In [0]:
%run /Workspace/Users/sandippatelaero@gmail.com/Databricks-Certified-Data-Engineer-Associate/Includes/Copy-Datasets

In [0]:
files = dbutils.fs.ls('dbfs:/mnt/demo-datasets/bookstore/orders-raw')
display(files)


In [0]:
%sql
select * from parquet.`dbfs:/mnt/demo-datasets/bookstore/orders-raw`


In [0]:
(spark.readStream.format('cloudFiles').option('cloudFiles.format','parquet').option('cloudFiles.schemaLocation','dbfs:/mnt/demo/checkpoints/multihop/schema/bronze2').load('dbfs:/mnt/demo-datasets/bookstore/orders-raw').createOrReplaceTempView('orders_raw_temp'))

In [0]:
%sql
create or replace temporary view orders_temp as (
  select *,current_timestamp() as ct,input_file_name() as file_name from orders_raw_temp
)

In [0]:
%sql
select * from orders_temp

In [0]:
spark.table('orders_temp').writeStream.format('delta').option('checkpointLocation','dbfs:/mnt/demo/checkpoints/multihop/output/bronze').outputMode('append').table('orders_bronze')


In [0]:
%sql
select count(*) as cnt from orders_bronze

In [0]:
load_new_data()

In [0]:
%sql
select count(*) as cnt from orders_bronze

In [0]:
(
    spark.read.format('json').load('dbfs:/mnt/demo-datasets/bookstore/customers-json').createOrReplaceTempView('customers_lookup')
)

In [0]:
%sql
select * from customers_lookup

In [0]:
spark.readStream.table('orders_bronze').createOrReplaceTempView('orders_bronze_temp')

In [0]:
%sql
create or replace temporary view orders_enriched_temp as (
  select order_id,quantity, a.customer_id,b.profile:first_name as first_name,cast(from_unixtime(order_timestamp,'yyyy-MM-dd HH:mm:ss') as timestamp) as order_timestamp,books from orders_bronze_temp as a inner join customers_lookup as b on a.customer_id = b.customer_id
  where quantity > 0

)

In [0]:
(
    spark.table('orders_enriched_temp').writeStream.format('delta').option('checkpointLocation','dbfs:/mnt/demo/checkpoints/multihop/output/silver').outputMode('append').table('orders_silver')
)

In [0]:
%sql
select * from orders_silver

In [0]:
%sql
select count(*) as cnt from orders_silver

In [0]:
load_new_data()

In [0]:
%sql
select count(*) as cnt from orders_silver

In [0]:
spark.readStream.table('orders_silver').createOrReplaceTempView('orders_silver_temp')

In [0]:
%sql
create or replace temp view daily_customer_books_temp as (
  select customer_id,first_name,date_trunc('DD',order_timestamp) as order_date,sum(quantity) as books_counts
  from orders_silver_temp
  group by customer_id,first_name,date_trunc('DD',order_timestamp)
)

In [0]:
%sql
select * from daily_customer_books_temp

In [0]:
spark.table('daily_customer_books_temp').writeStream.format('delta').outputMode('complete').option('checkpointLocation','dbfs:/mnt/demo/checkpoints/multihop/output/gold').trigger(availableNow=True).table('daily_customer_books')

In [0]:
%sql
select * from daily_customer_books

In [0]:
spark.readStream.table('daily_customer_books').createOrReplaceTempView('daily_customer_books_gold_temp')

In [0]:
%sql
select * from daily_customer_books_gold_temp

In [0]:
load_new_data(all=True)

In [0]:
for s in spark.streams.active:
    print('id',s.id)
    s.stop()
    s.awaitTermination()